# dec-iLQR
Location for Linear-Quadratic Regulator (LQR) and iLQR for interactive trajectory planning with applications to Decentralized Control.

References:
- [1] - [Dewolf - studywolf_control](https://github.com/studywolf/control/blob/master/studywolf_control/controllers/ilqr.py)
- [2] - [Fridovich-Keil - ilqgames](https://github.com/HJReachability/ilqgames/blob/master/python/dynamical_system.py)
- [3] - [Jackson, Howell - iLQR Tutorial](http://roboticexplorationlab.org/papers/iLQR_Tutorial.pdf)
- [4] - [Jackson - AL iLQR Tutorial](https://bjack205.github.io/papers/AL_iLQR_Tutorial.pdf)
- [5] - [Sears-Collins - Linear Quadratic Regulator Python Example](https://automaticaddison.com/linear-quadratic-regulator-lqr-with-python-code-example/)

In [1]:
import sys
sys.path.append('/mnt/c/Users/ZachJW/Documents/Grad/Research/dec-ilqr/dec-ilqr/')

import numpy as np
import matplotlib.pyplot as plt

from control import iLQR, LQR
from models import DoubleInt1dDynamics, DoubleInt2dDynamics
from models import CarDynamics, UnicycleDynamics, BicycleDynamics
from cost import ObstacleCost, ReferenceCost, AgentCost
from util import Point

In [2]:
%matplotlib ipympl
np.set_printoptions(precision=3, suppress=True)

In [3]:
f1 = plt.figure()
f2 = plt.figure()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### iLQR

In [5]:
dt = 0.1
N = 60
n_lqr_iter = 50
tol = 1e-3

# dynamics = DoubleInt1dDynamics(dt)
# x0 = np.array([2, 0])
# xf = np.array([0, 1])
# Q = np.diag([1, 1])
# R = np.eye(1) * 0.01

# dynamics = DoubleInt2dDynamics(dt)
# x0 = np.array([10, 10, 0, 0])
# xf = np.array([0, 0, 0, 0])
# Q = np.diag([1, 1, 0, 0])
# R = np.eye(2)

# dynamics = CarDynamics(dt)
# x0 = np.array([11, 11, np.pi/2])
# xf = np.array([0, 0, 0])
# Q = np.diag([1, 1, 0])
# R = np.eye(2) * 0.5

dynamics = UnicycleDynamics(dt)
x0 = np.array([0, 0, 0, 0])
xf = np.array([10, 10, 0, np.pi/4])
Q = np.diag([1, 1, 0, 0])
R = np.eye(2)

# dynamics = BicycleDynamics(dt)
# x0 = np.array([10, 10, np.pi/2, 0, 0])
# xf = np.array([0, 0, 0, 0, 0])
# Q = np.diag([1, 1, 0, 0, 0])
# R = np.eye(2)

Qf = np.eye(Q.shape[0]) * 1e3
reference = ReferenceCost(xf, Q, R, Qf)
obstacles = [ObstacleCost((0, 1), Point(3, 3), 2),
             ObstacleCost((0, 1), Point(9, 7), 2),
             ObstacleCost((0, 1), Point(3, 10), 3)]

REF_WEIGHT = 1.0
OBS_WEIGHT = 1e6
weights = [OBS_WEIGHT] * len(obstacles) + [REF_WEIGHT]
agent_cost = AgentCost(obstacles + [reference], weights)

In [6]:
ilqr = iLQR(dynamics, agent_cost, N=N)
X, U, Jf = ilqr.run(x0, n_lqr_iter, tol)

[run] 0/50	J: 212617
[run] 1/50	J: 66991.5	μ: 0.5	Δ: 0.5
[run] 2/50	J: 55656.4	μ: 0.125	Δ: 0.25
[run] 3/50	J: 54669.8	μ: 0.015625	Δ: 0.125
[run] 4/50	J: 54437.3	μ: 0.000976562	Δ: 0.0625
[run] 5/50	J: 54292.3	μ: 3.05176e-05	Δ: 0.03125
[run] 6/50	J: 35469	μ: 0	Δ: 0.015625
[run] 7/50	J: 14171.2	μ: 0	Δ: 0.0078125
[run] 8/50	J: 7350.44	μ: 0	Δ: 0.00390625
[run] 9/50	J: 7104.33	μ: 0	Δ: 0.00195312
[run] 10/50	J: 7023.41	μ: 0	Δ: 0.000976562
[run] 11/50	J: 3756.43	μ: 0	Δ: 0.000488281


In [25]:
# plt.figure(f1)
ilqr.plot(X, Jf, False, surface_plot=True, axis=(-1, 11, -1, 11), log_colors=True)

#### Visualizing the Cost Surface

In [22]:
obstacles = [ObstacleCost((0, 1), Point(3, 3), 2),
             ObstacleCost((0, 1), Point(9, 7), 2),
             ObstacleCost((0, 1), Point(3, 10), 3)]

xf = np.array([10, 10, 0, np.pi/4])
Q = np.diag([1, 1, 0, 0])
Qf = np.eye(Q.shape[0])
R = np.zeros((2,2))
ref = ReferenceCost(xf, Q, R, Qf)

OBS_WEIGHT = 100
REF_WEIGHT = 1.0

weights = [OBS_WEIGHT] * len(obstacles) + [REF_WEIGHT]
costs = obstacles + [ref]

agent_cost = AgentCost(costs, weights)
plt.figure(f2), plt.clf()
agent_cost.plot(True, x0=np.zeros(4), axis=(0, 11, 0, 11), log_colors=False)

#### Validating Obstacle Quadraticization

In [6]:
self = agent_cost
x = np.array([5.5, 5.1, 0, 0])
u = np.array([0, 0])
L = []
L_x = []
L_u = []
L_xx = []
L_uu = []
L_ux = []

for cost, weight in zip(self.costs, self.weights):
    L.append(cost(x, u) * weight)
    L_xi, L_ui, L_xxi, L_uui, L_uxi = cost.quadraticize(x, u)
    L_x.append(weight * L_xi)
    L_u.append(weight * L_ui)
    L_xx.append(weight * L_xxi)
    L_uu.append(weight * L_uui)
    L_ux.append(weight * L_uxi)
L_xx

[array([[2., 0., 0., 0.],
        [0., 2., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 array([[ 1849.141,   754.293,     0.   ,     0.   ],
        [  754.293, -1771.464,     0.   ,     0.   ],
        [    0.   ,     0.   ,     0.   ,     0.   ],
        [    0.   ,     0.   ,     0.   ,     0.   ]])]

### LQR

In [ ]:
N = 60
dt = 0.1

dynamics = DoubleInt1dDynamics(dt)
x0 = np.array([10, 0])
xf = np.array([0, 0])
Q = np.diag([1, 1])
R = np.eye(1)

# dynamics = DoubleInt2dDynamics(dt)
# x0 = np.array([1, 1, 0, 0])
# xf = np.array([0, 0, 0, 0])
# Q = np.eye(4)
# R = np.eye(2)

# dynamics = CarDynamics(dt)
# x0 = np.array([5.0, 5.0, np.pi/2])
# xf = np.array([0, 0, 0])
# Q = np.diag([1, 1, 0])
# R = np.eye(2) * 0.01

# dynamics = UnicycleDynamics(dt)
# x0 = np.array([-1, -1, 0, np.pi/3])
# xf = np.array([0, 0, 0, 0])
# Q = np.diag([1, 1, 0, 0])
# R = np.eye(2)

Qf = np.eye(Q.shape[0]) * 1e3
quad_cost = ReferenceCost(xf, Q, R, Qf)
lqr = LQR(dynamics, quad_cost, N=N)
X, U, J = lqr.run(x0)
lqr.plot(X, J)